In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler #Scaler

print (sklearn.__version__)
import site
print (site.getsitepackages())

0.18.dev0
['/home/ellfae/anaconda3/envs/py35/lib/python3.5/site-packages']


In [30]:
# load the data
df_train = pd.read_csv('train.csv/train.csv')
df_test = pd.read_csv('test.csv/test.csv')

#load the pickle where extra computer features are stored
#my_features = pickle.load(open("sums.pickle", "rb"))

In [31]:
feature_to_clean = ['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0', 'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46', 'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3', 'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3', 'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3', 'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3',
                   'ind_var29_0', 'ind_var29', 'ind_var13_medio', 'ind_var18', 'ind_var26', 
                     'ind_var25', 'ind_var32', 'ind_var34', 'ind_var37', 'ind_var39', 'num_var29_0', 
                     'num_var29', 'num_var13_medio', 'num_var18', 'num_var26', 'num_var25', 'num_var32', 
                     'num_var34', 'num_var37', 'num_var39', 'saldo_var29', 'saldo_medio_var13_medio_ult1', 
                     'delta_num_reemb_var13_1y3', 'delta_num_reemb_var17_1y3', 'delta_num_reemb_var33_1y3', 
                     'delta_num_trasp_var17_in_1y3', 'delta_num_trasp_var17_out_1y3', 
                     'delta_num_trasp_var33_in_1y3', 'delta_num_trasp_var33_out_1y3',
                   'delta_imp_reemb_var33_1y3',  'imp_reemb_var17_hace3', 'imp_reemb_var33_ult1', 'imp_trasp_var17_in_hace3', 'num_reemb_var17_hace3', 'num_reemb_var33_ult1', 'num_trasp_var17_in_hace3']



df_train.drop(feature_to_clean, axis=1, inplace=True)
df_test.drop(feature_to_clean, axis=1, inplace=True)

## Add binary features for each value
1. here I want to add additional columns for each value in each feature.
2. For now I will just try add add "1" for where "0" is found

In [32]:
features_extraction = ['num_var13_corto', 'num_var13_corto_0', 'num_meses_var12_ult3', 'num_meses_var13_corto_ult3',
                      'num_meses_var39_vig_ult3', 'num_meses_var5_ult3','num_var24_0','num_var12','var36',
                      'num_var5','num_var5_0','num_var12_0','num_var13','num_var13_0','num_var42','num_var4',
                      'num_var42_0','num_var30','num_var39_0','num_var41_0']


In [33]:
#add a binary new feature for each category in each of the features extraction
'''
def add_binary_features(cols,cat):
    for value in cols:
        if value == cat:
            return 1
        else:
            return 0 

tracker = 1
for f in features_extraction:
    categories = np.intersect1d( df_test[f] , df_train[f])
    #train_categories = df_train[f].unique()
    for cat in categories:
        df_train[f+"_"+str(cat)] = df_train[[f]].apply(add_binary_features, args=(cat,), axis=1)
        df_test[f+"_"+str(cat)] = df_test[[f]].apply(add_binary_features, args=(cat,), axis=1)
    
    print "Done with " + str(tracker)
    
    tracker+=1
    
# TODO: drop the features
df_train.drop(features_extraction, axis=1, inplace=True)
df_test.drop(features_extraction, axis=1, inplace=True)
'''

'\ndef add_binary_features(cols,cat):\n    for value in cols:\n        if value == cat:\n            return 1\n        else:\n            return 0 \n\ntracker = 1\nfor f in features_extraction:\n    categories = np.intersect1d( df_test[f] , df_train[f])\n    #train_categories = df_train[f].unique()\n    for cat in categories:\n        df_train[f+"_"+str(cat)] = df_train[[f]].apply(add_binary_features, args=(cat,), axis=1)\n        df_test[f+"_"+str(cat)] = df_test[[f]].apply(add_binary_features, args=(cat,), axis=1)\n    \n    print "Done with " + str(tracker)\n    \n    tracker+=1\n    \n# TODO: drop the features\ndf_train.drop(features_extraction, axis=1, inplace=True)\ndf_test.drop(features_extraction, axis=1, inplace=True)\n'

## Add Sum of 0's

In [34]:
features = [f for f in df_train.columns if f != 'TARGET' and f!='ID' ]
#df_train['sum_of_0'] = my_features['sum_of_0']
def calculate_zeros(cols):
    sum_array = [] 
    for value in cols:
        if value == 0:
            sum_array.append(value)
    return len(sum_array)

df_train['sum_of_0'] = df_train[features].apply(calculate_zeros, axis=1)
df_test['sum_of_0'] = df_test[features].apply(calculate_zeros, axis=1)

In [35]:
#prepare data for model
y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

In [36]:
id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

In [37]:
len_train = len(X_train)
len_test = len(X_test)

#scale data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

# apply same transformation to test data
X_test = scaler.transform(X_test)

In [58]:
# classifier
clf = MLPClassifier(hidden_layer_sizes=(32,100,32), activation='relu', 
     
     learning_rate_init=0.001, max_iter = 5000, random_state = 1235, 
     learning_rate='adaptive',
                   verbose=True, batch_size=200 )

In [59]:
# fitting
clf.fit(X_train, y_train)

Iteration 1, loss = 0.16878023
Iteration 2, loss = 0.14171645
Iteration 3, loss = 0.13854827
Iteration 4, loss = 0.13694234
Iteration 5, loss = 0.13560914
Iteration 6, loss = 0.13525031
Iteration 7, loss = 0.13401269
Iteration 8, loss = 0.13341853
Iteration 9, loss = 0.13272897
Iteration 10, loss = 0.13212703
Iteration 11, loss = 0.13139744
Iteration 12, loss = 0.13121433
Iteration 13, loss = 0.13026843
Iteration 14, loss = 0.13015638
Iteration 15, loss = 0.12919309
Iteration 16, loss = 0.12856298
Iteration 17, loss = 0.12793116
Iteration 18, loss = 0.12781769
Iteration 19, loss = 0.12770603
Iteration 20, loss = 0.12653349
Iteration 21, loss = 0.12633755
Iteration 22, loss = 0.12550075
Iteration 23, loss = 0.12476694
Iteration 24, loss = 0.12462755
Iteration 25, loss = 0.12357198
Iteration 26, loss = 0.12331921
Iteration 27, loss = 0.12310142
Iteration 28, loss = 0.12195190
Iteration 29, loss = 0.12165910
Iteration 30, loss = 0.12090958
Iteration 31, loss = 0.12062276
Iteration 32, los

MLPClassifier(activation='relu', algorithm='adam', alpha=0.0001,
       batch_size=200, beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(32, 100, 32),
       learning_rate='adaptive', learning_rate_init=0.001, max_iter=5000,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5,
       random_state=1235, shuffle=True, tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [60]:
# prediction
y_pred= clf.predict_proba(X_test)[:,1]
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

Overall AUC: 0.906017580468


In [61]:
submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission61.csv", index=False)

In [42]:
'''
ADDITIONAL FUNCTIONS FOR FEATURE ENGINEERING
==============================================================
import pickle
to_save = {
    "sum_of_0":df_train['sum_of_0']
}
pickle.dump(to_save, open('my_features.pickle','a'))
===============================================================
#NAME: ADD SUM OF 0'S
# select on the features, leaving out the 'TARGET' feature


features = [f for f in df_train.columns if f != 'TARGET' and f!='ID' ]

def calculate_zeros(cols):
    sum_array = [] 
    for value in cols:
        if value == 0:
            sum_array.append(value)
    return len(sum_array)

df_train['sum_of_0'] = df_train[features].apply(calculate_zeros, axis=1)

===============================================================

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1, len(c)):
        if np.array_equal(v, df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

================================================================

# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)
================================================================
#fix nationality

def fix_nationality(nat):
    for value in nat:
        if value == -999999:
            return 2
        else:
            return value

df_train['var3'] = df_train[['var3']].apply(fix_nationality, axis=1)
df_test['var3'] = df_test[['var3']].apply(fix_nationality, axis=1)


=================================================================
# Add feature selection

#Extract 49 features from all the features

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf_0 = ExtraTreesClassifier(random_state=1729)
selector = clf_0.fit(X_train, y_train)

fs = SelectFromModel(selector, prefit=True)

X_train = fs.transform(X_train)
X_eval = fs.transform(X_eval)
X_test = fs.transform(X_test)
=================================================


'''

'\nADDITIONAL FUNCTIONS FOR FEATURE ENGINEERING\n==============================================================\nimport pickle\nto_save = {\n    "sum_of_0":df_train[\'sum_of_0\']\n}\npickle.dump(to_save, open(\'my_features.pickle\',\'a\'))\n===============================================================\n#NAME: ADD SUM OF 0\'S\n# select on the features, leaving out the \'TARGET\' feature\n\n\nfeatures = [f for f in df_train.columns if f != \'TARGET\' and f!=\'ID\' ]\n\ndef calculate_zeros(cols):\n    sum_array = [] \n    for value in cols:\n        if value == 0:\n            sum_array.append(value)\n    return len(sum_array)\n\ndf_train[\'sum_of_0\'] = df_train[features].apply(calculate_zeros, axis=1)\n\n===============================================================\n\n# remove duplicated columns\nremove = []\nc = df_train.columns\nfor i in range(len(c)-1):\n    v = df_train[c[i]].values\n    for j in range(i+1, len(c)):\n        if np.array_equal(v, df_train[c[j]].values):\n        